* Adicione o conjunto de dados Mall Customer Segmentation Data ao notebook da aula.

* Efetue uma exploração inicial do conjunto (uma boa biblioteca para isso é a pandas-profiling, vale a pena usá-la).

* Aplique o modelo k-means para obter clusters de clientes de diferentes características. Identifique que tipos de clientes os clusters conseguem discriminar.

Dica: nem todas as variáveis precisam ser utilizadas para que se atinja uma separação adequada e interpretável.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
import yellowbrick.cluster
from copy import deepcopy
import seaborn as sns 
sns.set()

from pandas_profiling import ProfileReport

In [ ]:
mall_customers = pd.read_csv('../input/customer-segmentation-tutorial-in-python/Mall_Customers.csv')

## Analise

In [ ]:
mall_report = ProfileReport(mall_customers, title='Mall Report')

In [ ]:
mall_report

In [ ]:
plt.plot('CustomerID', 'Annual Income (k$)', data=mall_customers)
plt.show()

O report levantou o alertar que "CustomerID" e "Annual Income" possui alta correlação. Provavelmente essa correlação foi forçada sendo o customerID fornecido de acordo com a ordem de "Annual Income (k$)".  
Idade e valor gasto também tem uma correlação. Aparentemente quanto maior a idade menor o gasto.  
Não há valores faltantes. 
Há um número levemente maior de mulheres na lista de clientes.

Uma vez que há um número maior de mulheres a coluna de Gender será convertida para uma coluna booleana com o nome de "Woman" e a coluna CustomerID será considerar index

In [ ]:
mall_customers.set_index('CustomerID', inplace=True)
mall_customers['Woman'] = mall_customers['Gender']=='Female'
mall_customers.drop('Gender', axis=1, inplace = True)

In [ ]:
mall_customers.describe()

Para treinar o modelo será normalizado os dados 

In [ ]:
mall_customers

In [ ]:
mall_customers_normalized = (mall_customers - mall_customers.mean())/mall_customers.std()

### Selecionando Número de Clusters

In [ ]:
def plot_inercia_per_k (X, k_list, ax = None):

    ax = ax or plt.gca()
    
    kmeans_per_k = [KMeans(n_clusters=k, random_state=0).fit(X) for k in k_list]

    inercias = [m.inertia_ for m in kmeans_per_k]

    ax.plot(k_list, inercias,ls='-',marker='*')

    ax.set_xlabel('$k$')
    ax.set_ylabel('inércia')
    
    return kmeans_per_k

In [ ]:
_ = plot_inercia_per_k(mall_customers_normalized, range(2,20))

Não há inflexão clara

In [ ]:
def plot_silhueta_per_k(X, k_list, ax = None):

    ax = ax or plt.gca()
    
    kmeans_per_k = [KMeans(n_clusters=k, random_state=0).fit(X)
                    for k in k_list]

    silhuetas = [silhouette_score(X, m.labels_) for m in kmeans_per_k]

    ax.plot(k_list, silhuetas,ls='-',marker='*')

    ax.set_xlabel('$k$')
    ax.set_ylabel('silhueta média')
    
    return kmeans_per_k

In [ ]:
kmeans_per_k = plot_silhueta_per_k(mall_customers_normalized,range(2,20))

Existe um pico próximo a K=10 más para uma segmentação de mercado possuir 10 grupos não é útil.

In [ ]:
def plot_silhueta(X, k_list):
        
    n_lines = int(np.ceil(len(k_list)/2))    
    
    fig, ax = plt.subplots(n_lines, 2, figsize = (16,4*n_lines))

    for i in range(len(k_list)):

        m = KMeans(k_list[i])
        yellowbrick.cluster.silhouette_visualizer(m, X, show=False, ax=ax.ravel()[i]);

    fig.tight_layout()

In [ ]:
plot_silhueta(mall_customers_normalized, [2,3,4,5,6,7])

4 Clusters parece ser uma boa escolha para seoarar bem os grupos e ainda ter relevância de analise.

In [ ]:
def plot_cluster_attributes(df, k_list):
    fig, ax = plt.subplots(1,len(k_list), figsize=(20,5))
    
    for i in range(len(k_list)):

        df_tmp = deepcopy(df)
        df_tmp['ID'] = df.index
        kmeans_per_k = [KMeans(n_clusters=k, random_state=0).fit(df) for k in k_list]
        df_tmp['Cluster'] = kmeans_per_k[i].labels_
        df_tmp_melt = pd.melt(df_tmp.reset_index(),
                              id_vars=['ID', 'Cluster'],
                              value_vars=df.columns,
                              var_name='Attribute',
                              value_name='Value')

        sns.lineplot('Attribute', 'Value', hue='Cluster', data=df_tmp_melt, 
                     ax=ax[i], palette = sns.color_palette("husl", k_list[i]))

        ax[i].set_title(f'$k={k_list[i]}$')
        
    return kmeans_per_k

In [ ]:
plot_cluster_attributes(mall_customers_normalized, [4, 6])

Com 4 Clusters é possível ver uma separação de grupos

0 - Mulheres Jovens que possuem renda média e consomem muito.  
1 - Homens Jovens que que possuem renda elevada que consomem muito.  
2 - Homens mais velhos que possuem renda média e gastam pouco.  
3 - Mulheres mais velhas que possuem renda média e gastam pouco.  

porém a variabilidade desses clusters está bem alta principalmente com relação ao ganho. Essa métodologia não está separando bem os diferentes salários.

Com 10 clusters há uma segmentação de mercado muito grande na idade que pode ser interessante.

Para ter maior visão de mercado seria bom clusters com menos grupos poder separar entre pessoas que ganham muito e ganham pouco que gastam muito e que gastam pouco e idades. Portando vou re-fazer a análise retirando a coluna da woman.

In [ ]:
mall_customers_no_gender = mall_customers_normalized.drop('Woman', axis = 1)

In [ ]:
_ = plot_silhueta_per_k(mall_customers_no_gender,range(2,20))

In [ ]:
_  = plot_inercia_per_k(mall_customers_no_gender, range(2,20))

In [ ]:
plot_silhueta(mall_customers_no_gender, [2,3,4,5,6,7])

6 Clusters parece ser uma boa escolha para a separação dos dados devido a inercia e a silhueta

In [ ]:
k_means_6, k_means_7 = plot_cluster_attributes(mall_customers_no_gender, [6, 7])

Agora há uma melhor Separação entre os clusters com 6 temos:

0 - pessoas com idade média que ganham muito mas não gastam muito pouco.  
1 - Jovens adultos que ganham pouco mas gastam muito (provavelmente jovens que ainda moram com os pais e tem um emprego de meio periodo).  
2 - pessoas com idade avançada que ganham moderadamente e gastam de forma moderada.  
3 - Adultos que possuem um salário alto e gastam muito.  
4 - Jovens adultos que possuem uma salário moderado e gastam de forma moderada.  
5 - pessoas com idade mais avançada que ganham pouco e gastam pouco.  

Quando aumenta o número de clusters para 7 surge uma nova sub categoria de pessoas com idade média que ganham moderadamente e gastam moderadamente.

In [ ]:
mall_customers_no_gender['cluster'] = k_means_6.predict(mall_customers_no_gender)

In [ ]:
mall_customers_no_gender['cluster'].value_counts().sort_index()

Há um número mais ou menos homogêneo de pessoas em cada grupo sendo o mais frequente de pessoas com idade mais avançada que ganham de forma moderada e gastam de forma moderada.